In [ ]:
# !kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset --unzip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install kaggle

     -------------------------------------- 173.2/173.2 kB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 78.2/78.2 kB ? eta 0:00:00
  Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
     -------------------------------------- 163.4/163.4 kB 9.6 MB/s eta 0:00:00


In [12]:
# Download the dataset
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Define the dataset and download path
dataset = 'masoudnickparvar/brain-tumor-mri-dataset'
download_path = '../data/'

# Download the dataset
api.dataset_download_files(dataset, path=download_path, unzip=True)

print(f"Dataset downloaded and extracted to {download_path}")

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
Dataset downloaded and extracted to ../data/
